## 문서의 목적

본 문서는 [not MNIST](http://yaroslavvb.blogspot.kr/2011/09/notmnist-dataset.html) 데이터 셋을 활용하여, Tensorflow의 입력 데이터 셋을 만드는 과정을 서술하기 위해 작성되었습니다.


### Data download

not-MNIST 데이터 셋을 다운로드 한다. ubutu 또는 mac os 때는 아래 코드를 입력하며, windows에서는 아래 주소를 익스플로러에 넣은 후 저장한다.

http://yaroslavvb.com/upload/notMNIST/notMNIST_large.tar.gz

In [ ]:
!wget http://yaroslavvb.com/upload/notMNIST/notMNIST_large.tar.gz

다운로드 완료후 압축을 푸는 코드를 입력한다.

In [ ]:
!tar -xvf notMNIST_large.tar.gz

### Dataset의 구조

다운로드 받은 파일의 압축을 풀면 다음과 같은 파일의 구조를 가진다.

![](./images/directories.png)

위 파일 구조는 우리가 일반적으로 분류 문제를 푸는 문제를 만들 때, 가장 많이 사용하는 폴더 구조다. not MNIST는 A부터 J까지의 알파벳 문자를 그림으로 표현한 것이기 때문에, 개별 알파벳이 각 폴더에 들어가 있다. 예를 들면, A 폴더에 있는 글자들은 아래 같은 것들이 있다.

![](./notMNIST_large/A/ZXVyb2Z1cmVuY2UgYm9sZGl0YWxpYy50dGY=.png)
![](./notMNIST_large/A/ZXVyb2Z1cmVuY2UgbGlnaHQgaXRhbGljLnR0Zg==.png)

일반적으로 폴더 구조를 만들때는 class를 폴더명으로, 각 class의 이미지를 해당 폴더의 하위 파일로 저장하면 된다.

### npy 파일 만들기

각 폴더명에는 class를 폴더 별로 파일의 저장을 완료하면 해당 폴더에서 파일들을 호출하여 numpy 파일로 저장하여야 한다. 현재 파일들은 흑백으로 하나의 채널을 가진 이미지 파일로 이미지 사이즈를 기반으로 가로 세로 데이터를 가진다고 볼 수 있다. 예를 들면 아래 그림의 numpy 구성은 아래처럼 호출할 수 있다.

![](./notMNIST_large/A/ZXVyb2Z1cmVuY2UgbGlnaHQgaXRhbGljLnR0Zg==.png)


#### image의 numpy loading

In [1]:
import matplotlib.image as mpimg
import numpy as np

In [2]:
img_dir = "notMNIST_large/A/ZXVyb2Z1cmVuY2UgbGlnaHQgaXRhbGljLnR0Zg==.png"

a_imag = mpimg.imread(img_dir)
a_imag.shape, a_imag.dtype

((28, 28), dtype('float32'))

#### image의 numpy 구성

In [3]:
a_imag

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.00392157, 0.01568628, 0.        ,
        0.6313726 , 0.6156863 , 0.        , 0.00784314, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.01568628, 0.        , 0.38431373,
        1.        , 0.8509804 , 0.02352941, 0.        , 0.00392157,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.0117

#### dict type으로 numpy feature와 label을 저장

각 이미지에서 ndarray를 추출하여 dict에 저장한다

In [4]:
import os

dataset_dir = "notMNIST_large"

classes = []
labels = []
features = []
for root, dirs, files in os.walk(dataset_dir, topdown=False):
    for name in dirs:
        classes.append(name)

for class_name in classes: 
    root_dir = os.path.join(dataset_dir, class_name)
    for name in os.listdir(root_dir):
        img_path = os.path.join(root_dir, name)
        try:
            img_ndarray = mpimg.imread(img_path)
            features.append(img_ndarray)
            labels.append(class_name)
        except OSError as e:
            print(img_path)
            print(e)
        except ValueError as e:
            print(img_path)
            print(e)


notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png
read past end of file
notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png
invalid PNG header
notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png
read past end of file
notMNIST_large/D/VHJhbnNpdCBCb2xkLnR0Zg==.png
read past end of file
notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png
read past end of file


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(classes)
print(le.classes_)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']


In [6]:
labels_int = le.transform(labels)
from sklearn import preprocessing
ohe = preprocessing.OneHotEncoder()
ohe.fit(labels_int.reshape(-1, 1))
labels_one_hot = ohe.transform(labels_int.reshape(-1, 1))

In [7]:
labels_one_hot.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [8]:
data = {}
data["features"] = np.asarray(features)
data["labels"] = np.asarray(labels_one_hot.toarray())

In [9]:
data["features"].shape

(529114, 28, 28)

In [10]:
data["labels"].shape

(529114, 10)

In [11]:
train_set = {}
test_set = {}

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
for train_index, test_index in sss.split(data["features"] , data["labels"]):
    train_set["features"]  = data["features"][train_index]
    train_set["labels"] = data["labels"][train_index]
    
    test_set["features"] = data["features"][test_index]
    test_set["labels"] = data["labels"][test_index]


In [13]:
train_set["features"].shape, train_set["labels"].shape

((264557, 28, 28), (264557, 10))

In [14]:
test_set["features"].shape, test_set["labels"].shape

((264557, 28, 28), (264557, 10))

#### dictype의 persisence
저장된 dict type을 npy 파일로 persisence 한다.

In [15]:
np.save("notMNIST_large.npy",data)
np.save("notMNIST_large_train.npy",train_set)
np.save("notMNIST_large_test.npy",test_set)

만약 본인의 pc가 MAC이라면 아래 코드를 사용해야한다

In [16]:
!pip install zodbpickle

    100% |████████████████████████████████| 337kB 111kB/s ta 0:00:01
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
from zodbpickle import pickle # for Mac OS

with open('notMNIST_large.npy', 'wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('notMNIST_large_train.npy', 'wb') as f:
    pickle.dump(train_set, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('notMNIST_large_test.npy', 'wb') as f:
    pickle.dump(test_set, f, protocol=pickle.HIGHEST_PROTOCOL)